# 重線形回帰に関するコード

このコードは次のサイトの記述とコードを参考にしています。
https://www.kaggle.com/aakashns/pytorch-basics-linear-regression-from-scratch

## 準備

In [ ]:
import numpy as np
import torch

0次元テンソル(スカラー）の変数を作ってみます。

In [ ]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

中身を表示してみます。

In [ ]:
print('x = ', x)
print('w = ', w)
print('b = ', b)

テンソルの演算(スカラー演算)をしてみます。

In [ ]:
y = w * x + b
print('y = ', y)

## PyTorchにおける自動微分

PyTorchの自動微分計算機能を使ってみます。ここでは、与えられた$y = w*x+b$ に対して、$x = 3$のときの$dy/dw$と$dy/db$を計算します。

In [ ]:
y.backward()

導関数値を表示します。自分で計算した導関数値と一致するか、確認してみてください。

In [ ]:
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

## PyTorchにおけるブロードキャスティング

In [ ]:
a = torch.tensor(2.0)
torch.sigmoid(a)

関数の適用は、入力テンソルの各要素ごとに適用される

In [ ]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
torch.sigmoid(X)

1次元テンソルを入力とする関数を定義してみる。 
$$
test_func(x) = x_1 + x_2 + x_3 + 1
$$

"@" は行列(テンソル)の積を表す(ちなみに"*"は要素ごとの積)

In [ ]:
torch.ones(3,1) # 3 x 1 テンソル

In [ ]:
def test_func(x):
    return x @ torch.ones(3, 1) + 1

a = torch.tensor([1.0, 2.0, 3.0])
test_func(a)

test_funcをXに適用するとXの各行にtest_funcが適用されて、その結果が列ベクトルとして返される(ブロードキャスティング)
ブロードキャスティングは、前回のニューラルネットワークモデルをミニバッチに適用するときにも自動的に適用されたいる

In [ ]:
test_func(X)

## 重線形回帰をしてみよう

次のデータはりんごとオレンジの収穫量と３つの要因（温度、降雨量、湿度）の関係のデータである。(表データは外部サイトのpngファイルにリンクしています)
<img src="https://i.imgur.com/lBguUV9.png">

## 以降で定義する重回帰モデル

### yeild_apple  = w1 * temp + w2 * rainfall + w3 * humidity + b

## 入力データ(温度・降雨量・湿度)

In [ ]:
# Input (temp, rainfall, humidity)
X = torch.Tensor(
    [[73, 67, 43], 
    [91, 88, 64], 
    [87, 134, 58], 
    [102, 43, 37], 
    [69, 96, 70]])
print(X)

## 収穫量のデータ(りんご)

In [ ]:
Y = torch.Tensor(
[[56], 
 [81], 
 [119], 
 [22], 
 [103]]
)
print(Y)

## 重み行列とバイアスを準備する

In [ ]:
# Weights and biases
w = torch.randn(3, 1, requires_grad=True)
b = torch.randn(1, requires_grad=True)
print(w)
print(b)

## 重回帰モデル
$$
\hat y = x w + b
$$
ここで、
* $x = (x_1, x_2, x_3) \in \mathbb{R}^3$の要素はそれぞれ温度, 降雨量, 湿度を表すデータ
* $w = (w_1, w_2, w_3)^T\in \mathbb{R}^3$は学習パラメータ
* $b \in \mathbb{R}$も学習パラメータ

In [ ]:
def model(x):
    return x @ w + b # @は行列の積を表す

## 入力$X$に対して予測値を計算してみる


初期パラメータはランダムなのでむちゃくちゃな予測値となる

In [ ]:
pred = model(X)
print(pred)

上のpred = model(X)は想定している入力の次元(3次元ベクトル)と実際の入力($5 \times 3$行列)に違いがあるように思えるが、PyTorchのブロードキャストが自動的に適用されるため、$X$の各行に対してmodelを適用した結果得られる列ベクトルが返り値となる

In [ ]:
print(Y)

## 損失関数を定義する

$$
\hat Y  = (\hat Y_1, \hat Y_2, \ldots, \hat Y_M)
$$

$$
Y  = (Y_1, Y_2, \ldots, Y_M)
$$

$$
mse(Y, \hat Y) = \frac 1 M \sum_{i=1}^M (Y_i - \hat Y_i)^2
$$

In [ ]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel() ## numel()はテンソルに含まれる要素数を返す

## 損失値を計算してみる

In [ ]:
loss = mse(pred, Y)
print(loss)

## 勾配ベクトルを計算する

In [ ]:
loss.backward()

In [ ]:
print(w)
print(w.grad)

In [ ]:
print(b)
print(b.grad)

## 勾配法を利用して$W,b$を調整する。

In [ ]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

preds = model(X)
loss = mse(preds, Y)
loss.backward()
print(loss)

##  学習ループを作成する

In [ ]:
w = torch.randn(3, 1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

for i in range(10):
    preds = model(X)
    loss = mse(preds, Y)
    loss.backward()
    print(i, loss.item())
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

## 重回帰予測出力と$Y$を比較する

In [ ]:
model(X)

In [ ]:
Y

-----------------

## 演習問題１

上記のコードを実行しつつ、理解せよ。この問題についてはレポートによる報告は必要ない。


## 演習問題2

「準備」のところでみたような自動微分計算の実例を自分で作り報告せよ(自分で検算も行っておくこと）。


## 演習問題3

次の2点についてコードを修正し、実行し結果をレポートに報告せよ。

* 学習率 $10^{-5}$を変更するとどのような結果が得られるか。
* 上記の学習ループはループ数が100である。ループ数を多くすると結果はどうなるか報告せよ。



## 演習問題4

* なるべく訓練誤差が小さくなるように学習を行い、パラメータを設定せよ。その予測モデルで、

入力: 温度80F, 降雨量80mm, 湿度80% 

の場合の予測値を計算し、報告せよ。



-----------------